In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
dpi = 300
savepath = os.getcwd() + '/Figures_paper/'

In [ ]:
def select_models(ds, varx1, vary, check, varx2=None):
    '''
    Check is either 'model', or 'scenario'
    Find models for which both variables (varx1, vary) are available
    Input is an xarray dataset (CMIP5ds, CMIP5ds_LF, CMIP6ds, CMIP6ds_LF). 
    If you want to check the scenarios per model, already select the model in the input: eg. CMIP5ds.sel(model=mod)
    It returns the model/sce names of models/sces that have data for all three variables and returns the new dataset.
    '''
    x1 = ds[varx1].dropna(check,'all')[check].values
    y = ds[vary].dropna(check,'all')[check].values
    
    if varx2 == None:
        check_list = np.sort(list(set(x1)&set(y)))

    if varx2 is not None:
        x2 = ds[varx2].dropna(check,'all')[check].values
        check_list = np.sort(list(set(x1)&set(x2)&set(y)))
    
    if check == 'model':
        ds_new = ds.sel(model=check_list)
        
    elif check == 'scenario':
        ds_new = ds.sel(scenario=check_list)
    
    return check_list, ds_new

## Figure 1. Spatial zos and projections zos
- Regional plots for intermediate emission scenario RCP4.5/SSP2-4.5 for CMIP5 and CMIP6. (panel a - b)
- Time-series for low, intermediate, and high emission scenarios for CMIP5 and CMIP6. (panel c - e)

In [ ]:
CMIP5_spatial_zos = xr.open_dataset('Data/CMIP5_spatial_zos.nc')
CMIP6_spatial_zos = xr.open_dataset('Data/CMIP6_spatial_zos.nc')

CMIP5_time_series = xr.open_dataset('Data/CMIP5_time_series_smoothed.nc')
CMIP6_time_series = xr.open_dataset('Data/CMIP6_time_series_smoothed.nc')

In [ ]:
def define_area(reg):
    '''Provides box coordinates given a region name'''
    
    if reg == 'knmi14_reg':
        lon_min, lon_max = -3.5, 7.5
        lat_min, lat_max = 51, 60
    elif reg == 'LS':
        lon_min, lon_max = -61.5, -34.5
        lat_min, lat_max = 50.5, 64.5
    elif reg == 'GS':
        lon_min, lon_max = -10.5,15.5
        lat_min, lat_max  = 65.5, 80.5 
    elif reg == 'IS':
        lon_min, lon_max = -34.5,0.5
        lat_min, lat_max  = 50.5, 67.5 
    else:
        print('Region not defined, specify region in function.')

    return lon_min, lon_max, lat_min, lat_max


In [ ]:
# Compute and plot multi-model ensemble median for intermediate scenario RCP4.5 / SSP2-4.5
# Panel a-b Figure 1
concat_sce_45 = xr.concat([CMIP5_spatial_zos.sel(scenario='rcp45',time=slice(2081,2101)).median(dim=['time','model']), 
                           CMIP6_spatial_zos.sel(scenario='ssp245',time=slice(2081,2101)).median(dim=['time','model'])],dim='scenario')


# Plotting extent
lon_min, lon_max, lat_min, lat_max = -15, 30, 30, 67

fig, ax = plt.subplots(1,2, figsize=(16,6),
            subplot_kw=dict(projection=ccrs.Orthographic(5, 52), facecolor="lightgray"),
            transform=ccrs.PlateCarree(), dpi=dpi)
plt.subplots_adjust(wspace=0.02, bottom=0.15)

titles = ['a) CMIP5 RCP4.5', 'b) CMIP6 SSP2-4.5']

for i in range(2):
    cb = ax[i].pcolormesh(concat_sce_45.lon,concat_sce_45.lat,concat_sce_45.isel(scenario=i).spatial_zos.values,transform = ccrs.PlateCarree(),vmin=-25,vmax=25,cmap='cmo.balance')
    ax[i].add_feature(cfeature.LAND, facecolor='lightgrey', zorder=1) 
    ax[i].coastlines()   
    ax[i].set_title(titles[i],fontsize=20)
    ax[i].set_extent([lon_min, lon_max, lat_min, lat_max])

    gl = ax[i].gridlines(color='grey', linestyle='-', draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlocator = mticker.FixedLocator([-20,0,20])
    gl.xlabel_style = {'size': 18, 'color':'black'}
    gl.ylabel_style = {'size': 18, 'color':'black'}

    lon_min_b, lon_max_b, lat_min_b, lat_max_b = define_area('knmi14_reg')
    x, y = ([lon_min_b, lon_min_b, lon_max_b, lon_max_b, lon_min_b], 
        [lat_max_b, lat_min_b, lat_min_b, lat_max_b, lat_max_b])

    ax[i].plot(x, y, marker='o', color = 'k', linewidth=2.0, transform=ccrs.PlateCarree())

cax = fig.add_axes([.08,.124,1,.754])
cax.set_axis_off()

cbar = plt.colorbar(cb, ax = cax,ticks=[-20,-10, 0, 10,20],cmap='cmo.balance', orientation='vertical',ticklocation='auto',extend='both')
cbar.ax.tick_params(labelsize=18) 
cbar.set_label('ODSL change [cm]', size=18)

fig.savefig('Figures/Fig1a-b.pdf', bbox_inches='tight', dpi = 300)

In [ ]:
def make_barplot(ds, mip, perc, start_year, end_year):
    
    # Create Data Frame 
    col = [f'{mip.upper()} Scenario']
    for p in perc:
        col.append(f'Percentile: {p}')    
    summary_df = pd.DataFrame(columns=col) 
    
    if mip == 'cmip5':
        sces_m = ['rcp26','rcp45','rcp85']
    elif mip == 'cmip6':
        sces_m = ['ssp126','ssp245','ssp585']
    
    for idx,sce in enumerate(sces_m):
        
        # Select scenario and models that have zos available
        ds_mods = ds.sel(scenario=sce)
        ds_mods_list = ds_mods['zos'].dropna('model','all').model
        ds_mods = ds_mods.sel(model=ds_mods_list)
        print(f'# of models for {sce}, {mip}: {len(ds_mods.model.values)}')

        # Calculate zos for period of interest:
        zos = ds_mods.sel(time=slice(start_year, end_year)).mean(dim='time').zos

        # Append the percentile scores to a list val
        val = [sces_m[idx]]
        for p in perc:
            val.append(round(np.quantile(zos.values,p/100),1))
        summary_df.loc[idx] = val
    
    summary_df.set_index(col[0], inplace=True)
    summary_df = summary_df.T
        
    return summary_df

In [ ]:
def plot(ax,df,colors=None,vlines=False):
    mi = 0.6 # Max color intensity
    
    # Get some pastel shades for the colors
    if not(colors):
        colors = plt.cm.Reds(np.linspace(0, mi, len(df.index)))
        rowColours = colors
        
        # Expand the array
        ones = np.ones(len(df.columns))
        colors = colors[np.newaxis,:,:] * ones[:, np.newaxis, np.newaxis]
        
    elif colors=='linspace':
        colors1 = plt.cm.Reds(np.linspace(0, mi, len(df.index)))
        colors2 = plt.cm.Blues(np.linspace(0, mi, len(df.index)))
        colors = np.zeros([len(df.columns), len(df.index), 4])
        colors[::2] = colors2
        colors[1::2] = colors1
        
        rowColours = plt.cm.Greys(np.linspace(0, mi, len(df.index)))

    elif colors=='max':
        colors1 = plt.cm.Reds([0,0.3,0.6,0.6,0.3])
        colors2 = plt.cm.Blues([0,0.3,0.6,0.6,0.3])
        colors = np.zeros([len(df.columns), len(df.index), 4])
        colors[::2] = colors2
        colors[1::2] = colors1
        
        rowColours = plt.cm.Greys(np.linspace(0, mi, len(df.index)))

    # Start from white color
    colors[:,0,:] = 0
    
    index = np.arange(len(df.columns))
    bar_width = [0.1,0.2,0.4,0.4,0.2]

    # Initialize the vertical-offset for the stacked bar chart.
    y_offset = np.zeros(len(df.columns))

    # Plot bars and create text labels for the table
    cell_text = []
    for row in range(len(df.index)):
        ax.bar(index, 
               df.iloc[row]-y_offset, 
               bar_width[row], 
               bottom=y_offset, 
               color=colors[:,row,:])
        
        y_offset = df.iloc[row]
        cell_text.append(['%1.1f' % x for x in df.iloc[row]])
    
    ax.set_xlim(-0.5,index[-1]+1.5)

    # Visualize median by a white horizontal line
    ax.hlines(df.iloc[2,0], xmin=index[0]-0.7*bar_width[2], xmax=index[0]+0.7*bar_width[2], lw=2, color='white', zorder=5)
    ax.hlines(df.iloc[2,1], xmin=index[1]-0.7*bar_width[2], xmax=index[1]+0.7*bar_width[2], lw=2, color='white', zorder=5)

    return ax, colors

In [ ]:
# Plotting time-series median and 17% - 83% uncertainty range
# Panel c - e
fig, ax = plt.subplots(1,6,figsize=(21,6), gridspec_kw={'width_ratios': [8, 1, 8, 1, 8, 1]},sharey=True,dpi=dpi)
plt.subplots_adjust(wspace=0.02, bottom=0.15)

# Compute percentiles for all scenarios
df5 = make_barplot(CMIP5_time_series, 'cmip5', [5,17,50,83,95], 2081, 2101)
df6 = make_barplot(CMIP6_time_series, 'cmip6', [5,17,50,83,95], 2081, 2101)
df = pd.concat([df5,df6],axis=1)

for i, sce in enumerate(['26', '45', '85']):
    ax[i*2].set_ylim([-5,35])
    ax[i*2].set_xlim([1900,2100])
    ax[i*2].set_xlabel('Year', fontsize = 20)
    ax[i*2].tick_params(axis='both', labelsize= 18, which='major', pad=10)
    ax[i*2].axvspan(1900, 1950, alpha=0.1, color='grey')
    ax[i*2].grid(True,alpha=0.3)
    ax[i*2].set_yticks([0,10,20,30])

    CMIP5 = CMIP5_time_series.isel(scenario=i)
    CMIP6 = CMIP6_time_series.isel(scenario=i)
    
    C5, = ax[i*2].plot(CMIP5.time.values, np.nanmedian(CMIP5.zos.values,axis=0), color=plt.cm.Blues([0.6]), label='CMIP5 median',lw=2)
    C6, = ax[i*2].plot(CMIP6.time.values, np.nanmedian(CMIP6.zos.values,axis=0), color=plt.cm.Reds([0.6]), label='CMIP6 median',lw=2)

    C55 = ax[i*2].fill_between(CMIP5.time.values, np.nanpercentile(CMIP5.zos.values,17,axis=0), 
                np.nanpercentile(CMIP5.zos.values,83,axis=0), color=plt.cm.Blues([0.6]),alpha=0.25,
                label='CMIP5, 5-95 percentiles')

    C66 = ax[i*2].fill_between(CMIP6.time.values, np.nanpercentile(CMIP6.zos.values,17,axis=0), 
                np.nanpercentile(CMIP6.zos.values,83,axis=0), color=plt.cm.Reds([0.6]),alpha=0.25,
                label='CMIP6, 5-95 percentiles')

    C5, = ax[i*2].plot(CMIP5.time.values, np.nanmedian(CMIP5.zos.values,axis=0), color=plt.cm.Blues([0.6]), label='CMIP5 median',lw=2,zorder=5)
    C6, = ax[i*2].plot(CMIP6.time.values, np.nanmedian(CMIP6.zos.values,axis=0), color=plt.cm.Reds([0.6]), label='CMIP6 median',lw=2,zorder=5)

    # Plotting and selecting average percentiles over period (2081 - 2101) for this scenario
    df_sce = df.iloc[:,[i,i+3]]
    print(df_sce)
    ax[i*2+1].set_axis_off()
    ax[i*2+1] = plot(ax[i*2+1],df_sce,colors='max',vlines=False)

# Add y-label, titles and legend  
ax[0].set_ylabel('ODSL change (cm)', fontsize = 20)

ax[0].set_title(f'c) RCP2.6 / SSP1-2.6',fontsize=20)
ax[2].set_title(f'd) RCP4.5 / SSP2-4.5',fontsize=20)
ax[4].set_title(f'e) RCP8.5 / SSP5-8.5',fontsize=20)

ax[0].legend([(C5, C55), (C6, C66)], 
             [f"CMIP5 median and 17-83% range",f"CMIP6 median and 17-83% range"],
             loc=2, fontsize=16);

fig.savefig('Figures/Fig1c-e.pdf', bbox_inches='tight', dpi = 300)

## Figure 2 - 4. Linear regression results

In [ ]:
def plot_LR_results(ds, varx1, cmip, save=None):
    
    ds = select_models(ds, varx1, 'zos', 'model')[1]

    # open LR_results:
    LR_result_data = pd.read_csv(f'Results_LR/LR_{cmip}_{varx1}.csv')
    LR_result = LR_result_data.set_index('model')

    nrows = int(np.round(len(LR_result)/4+.499))
    ncols = 4

    # ========================== #
    ### Plot individual models ###
    # ========================== #
    fig, ax = plt.subplots(nrows, ncols, figsize = (20, nrows*3.5),sharex=True,sharey=True)
    ax = ax.ravel()

    Models = LR_result.index.values

    ODSL_mods, ODSL_fits, ODSL_alph, ODSL_beta = [],[],[],[]
    
    for i, mod_name in enumerate(Models):
        sces_in_mod = select_models(ds.sel(model=mod_name), varx1, 'zos', 'scenario')[0]
        ds_mod = ds.sel(model = mod_name)
        
        if cmip == 'CMIP5':
            end_hist = 2005
            nan_num = 95
            sce_names = ['rcp26','rcp45','rcp85']
            
        elif cmip == 'CMIP6':
            end_hist = 2014
            nan_num = 86
            sce_names = ['ssp126','ssp245','ssp585']
        
        pars = LR_result.loc[mod_name]

        ds_hist = ds_mod.sel(time=slice(1900,end_hist), scenario=sces_in_mod[0])  # select historical period for one sce
        ds_sces = ds_mod.sel(time=slice(end_hist,2100))                          # select future period for all sces

        ODSL_mod, varx1_mod = ds_hist['zos'].values, ds_hist[varx1].values

        for sce in sce_names:        
                ds_sce = ds_sces.sel(scenario=sce)
            
                zos = ds_sce.zos.values
                varx1_list = ds_sce[varx1].values
            
                if(np.isnan(zos).any()) or (np.isnan(varx1_list).any()):
                    ODSL_mod = np.append(ODSL_mod, np.ones(nan_num)*np.nan)
                    varx1_mod = np.append(varx1_mod, np.ones(nan_num)*np.nan)
                else:
                    ODSL_mod = np.append(ODSL_mod, zos)
                    varx1_mod = np.append(varx1_mod, varx1_list)

        
        ODSL_fit = pars['alpha'] + pars['beta'] * varx1_mod
        
        ODSL_mods.append(ODSL_mod)
        ODSL_fits.append(ODSL_fit)
        ODSL_alph.append(ODSL_fit - pars['beta'] * varx1_mod)
        ODSL_beta.append(pars['beta']*varx1_mod)

        ax[i].plot(ODSL_mod, label='CMIP Data')
        ax[i].plot(ODSL_fit - pars['beta'] * varx1_mod, c='g',label=r'$\alpha$')
        ax[i].plot(ODSL_fit - pars['alpha'], c='k',label=r'$\beta_1 \times$'f'{varx1}')
        ax[i].plot(ODSL_fit, label='Regression Model', c='r')
        
        if i % 4 == 0:
            ax[i].set_ylabel('ODSL anomaly (cm)', fontsize = 15)        
        ax[i].tick_params(axis='x', labelsize=14)
        ax[i].tick_params(axis='y', labelsize=15)
    
        if cmip == 'CMIP5':
            ax[i].set_xticks([0, 105, 200,295], ["Historic", "RCP2.6", "RCP4.5",'RCP8.5'],  horizontalalignment='left')
        elif cmip == 'CMIP6':
            ax[i].set_xticks([0, 114, 200,286], ["Historic", "SSP1-2.6", "SSP2-4.5",'SSP5-8.5'],  horizontalalignment='left')
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        textstr = r'$\alpha$' f' = {pars.alpha:.2f} \n' r'$\beta_1$' f' = {pars.beta:.2f}'
        if i == 0:
            ax[i].text(0.75, 0.95, textstr, transform=ax[i].transAxes, fontsize=14, verticalalignment='top', bbox=props)
            ax[i].legend(loc='upper left',fontsize=14)
        else:
            ax[i].text(0.05, 0.95, textstr, transform=ax[i].transAxes, fontsize=14, verticalalignment='top', bbox=props)
        
        ax[i].set_ylim([-10,50])
        ax[i].set_title(f'{mod_name} \n 'r'R$^2$'f'-score: {pars[1]:.2f}, RMSE: {pars.mse:.2f} cm', fontsize=16)
        ax[i].grid(True, alpha=0.3)
    
    for j in range(len(Models), ncols*nrows):
            ax[j].axis("off")
    
    for j in range(len(Models)-ncols, len(Models)):
        if cmip == 'CMIP5':
            ax[j].xaxis.set_tick_params(labelbottom=True, labelcolor="k")
            ax[j].set_xlabel('Time', fontsize=15)

        if cmip == 'CMIP6':
            ax[j].xaxis.set_tick_params(labelbottom=True, labelcolor="k")
            ax[j].set_xlabel('Time', fontsize=15)

    ax[0].legend(loc='upper left',fontsize=16)
    fig.tight_layout()
    
    if save == True:
        fig.savefig(f'Figures/LR/Individual_Models_LR_{cmip}_{varx1}.pdf',dpi=dpi)
    elif save:
        fig.savefig(f'Figures/{save}_IM.pdf',dpi=dpi)


    # ========================= #
    ### Plot ensemble average ###
    # ========================= #
    ODSL_mods_mean = np.nanmean(np.array(ODSL_mods),axis=0)
    ODSL_fits_mean = np.nanmean(np.array(ODSL_fits),axis=0)
    ODSL_alph_mean = np.nanmean(np.array(ODSL_alph),axis=0)
    ODSL_beta_mean = np.nanmean(np.array(ODSL_beta),axis=0)

    fig,ax = plt.subplots(1,1,figsize=(10,6))
    ax.plot(ODSL_mods_mean,         label='CMIP Data')
    ax.plot(ODSL_alph_mean, c='g',  label=r'$\alpha$')
    ax.plot(ODSL_beta_mean, c='k',  label=r'$\beta_1 \times$'f'{varx1}')
    ax.plot(ODSL_fits_mean, c='r',  label='Regression Model')
    ax.set_xlabel('Time', fontsize=16)
    ax.set_ylabel('ODSL change (cm)', fontsize = 16)

    
    if cmip == 'CMIP5':
        ax.set_xlim([0,390])
        ax.set_xticks([0, 105, 200,295], ["Historic", "RCP2.6", "RCP4.5",'RCP8.5'],  horizontalalignment='left')

    elif cmip =='CMIP6':
        ax.set_xlim([0,372])
        ax.set_xticks([0, 114, 200,286], ["Historic", "SSP1-2.6", "SSP2-4.5",'SSP5-8.5'],  horizontalalignment='left')

    ax.set_ylim([-10,40])
    ax.axvspan(0, 50, alpha=0.1, color='grey')
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y', labelsize=15)
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper left',fontsize=16)
    ax.set_title(f'{cmip} ensemble average \n Average RMSE: {np.mean(LR_result.mse.values):.2f} cm', fontsize=17)

    if save == True:
        fig.savefig(f'Figures/LR/Average_LR_{cmip}_{varx1}.pdf',dpi=dpi)
    elif save:
        fig.savefig(f'Figures/{save}.pdf',dpi=dpi)
    return

def plot_LR_results_multi(ds, varx1, varx2, cmip, save=None):
    
    ds = select_models(ds, varx1, 'zos', 'model', varx2)[1]

    LR_result_data = pd.read_csv(f'Results_LR/LR_{cmip}_{varx1}_{varx2}.csv')
    LR_result = LR_result_data.set_index('model')

    nrows = int(np.round(len(LR_result)/4+.499))
    ncols = 4

    # ========================== #
    ### Plot individual models ###
    # ========================== #
    fig, ax = plt.subplots(nrows, ncols, figsize = (20, nrows*3.5),sharex=True,sharey=True)
    ax = ax.ravel()

    Models = LR_result.index.values

    ODSL_mods, ODSL_fits, ODSL_alph, ODSL_beta1, ODSL_beta2 = [],[],[],[],[]
    
    for i, mod_name in enumerate(Models):
        sces_in_mod = select_models(ds.sel(model=mod_name), varx1, 'zos', 'scenario', varx2)[0]
        ds_mod = ds.sel(model = mod_name)
        
        if cmip == 'CMIP5':
            end_hist = 2005
            nan_num = 95
            sce_names = ['rcp26','rcp45','rcp85']
            
        elif cmip == 'CMIP6':
            end_hist = 2014
            nan_num = 86
            sce_names = ['ssp126','ssp245','ssp585']
        
        pars = LR_result.loc[mod_name]

        ds_hist = ds_mod.sel(time=slice(1900,end_hist), scenario=sces_in_mod[0])  # select historical period for one sce
        ds_sces = ds_mod.sel(time=slice(end_hist,2100))                          # select future period for all sces

        ODSL_mod, varx1_mod, varx2_mod = ds_hist['zos'].values, ds_hist[varx1].values, ds_hist[varx2].values

        for sce in sce_names:        
                ds_sce = ds_sces.sel(scenario=sce)
            
                zos = ds_sce.zos.values
                varx1_list = ds_sce[varx1].values
                varx2_list = ds_sce[varx2].values

                if(np.isnan(zos).any()) or (np.isnan(varx1_list).any()):
                    ODSL_mod = np.append(ODSL_mod, np.ones(nan_num)*np.nan)
                    varx1_mod = np.append(varx1_mod, np.ones(nan_num)*np.nan)
                    varx2_mod = np.append(varx2_mod, np.ones(nan_num)*np.nan)
                else:
                    ODSL_mod = np.append(ODSL_mod, zos)
                    varx1_mod = np.append(varx1_mod, varx1_list)
                    varx2_mod = np.append(varx2_mod, varx2_list)

        ODSL_fit = pars['alpha'] + pars['beta1'] * varx1_mod + pars['beta2'] * varx2_mod
        
        ODSL_mods.append(ODSL_mod)
        ODSL_fits.append(ODSL_fit)
        ODSL_alph.append(ODSL_fit - pars['beta1'] * varx1_mod - pars['beta2'] * varx2_mod)
        ODSL_beta1.append(pars['beta1']*varx1_mod)
        ODSL_beta2.append(pars['beta2']*varx2_mod)

        ax[i].plot(ODSL_mod, label='CMIP Data')
        ax[i].plot(ODSL_fit - pars['beta1'] * varx1_mod - pars['beta2'] * varx2_mod, c='g',label=r'$\alpha$')
        ax[i].plot(pars['beta1'] * varx1_mod, c='k',label=r'$\beta_1 \times$'f'{varx1}')
        ax[i].plot(pars['beta2'] * varx2_mod, c='orange',label=r'$\beta_2 \times$'f'{varx2}')
        ax[i].plot(ODSL_fit, label='Regression Model', c='r')
        
        if i % 4 == 0:
            ax[i].set_ylabel('ODSL anomaly (cm)', fontsize = 15)        
        ax[i].tick_params(axis='x', labelsize=14)
        ax[i].tick_params(axis='y', labelsize=15)
    
        if cmip == 'CMIP5':
            ax[i].set_xticks([0, 105, 200,295], ["Historic", "RCP2.6", "RCP4.5",'RCP8.5'],  horizontalalignment='left')
        elif cmip == 'CMIP6':
            ax[i].set_xticks([0, 114, 200,286], ["Historic", "SSP1-2.6", "SSP2-4.5",'SSP5-8.5'],  horizontalalignment='left')
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        textstr = r'$\alpha$' f' = {pars.alpha:.2f} \n' r'$\beta_1$' f' = {pars.beta1:.2f} \n' r'$\beta_2$' f' = {pars.beta2:.2f}'
        if i == 0:
            ax[i].text(0.75, 0.95, textstr, transform=ax[i].transAxes, fontsize=14, verticalalignment='top', bbox=props)
            ax[i].legend(loc='upper left',fontsize=14)
        else:
            ax[i].text(0.05, 0.95, textstr, transform=ax[i].transAxes, fontsize=14, verticalalignment='top', bbox=props)
        
        ax[i].set_ylim([-10,50])
        ax[i].set_title(f'{mod_name} \n 'r'R$^2$'f'-score: {pars[1]:.2f}, RMSE: {pars.mse:.2f} cm', fontsize=16)
        ax[i].grid(True, alpha=0.3)
    
    for j in range(len(Models), ncols*nrows):
            ax[j].axis("off")
    
    for j in range(len(Models)-ncols, len(Models)):
        if cmip == 'CMIP5':
            ax[j].xaxis.set_tick_params(labelbottom=True, labelcolor="k")
            ax[j].set_xlabel('Time', fontsize=15)

        if cmip == 'CMIP6':
            ax[j].xaxis.set_tick_params(labelbottom=True, labelcolor="k")
            ax[j].set_xlabel('Time', fontsize=15)

    ax[0].legend(loc='upper left',fontsize=16)
    fig.tight_layout()
    
    if save == True:
        fig.savefig(f'Figures/LR/Individual_Models_LR_{cmip}_{varx1}_{varx2}.pdf',dpi=dpi)
    elif save:
        fig.savefig(f'Figures/{save}_IM.pdf',dpi=dpi)



    # ========================= #
    ### Plot ensemble average ###
    # ========================= #
    ODSL_mods_mean = np.nanmean(np.array(ODSL_mods),axis=0)
    ODSL_fits_mean = np.nanmean(np.array(ODSL_fits),axis=0)
    ODSL_alph_mean = np.nanmean(np.array(ODSL_alph),axis=0)
    ODSL_beta1_mean = np.nanmean(np.array(ODSL_beta1),axis=0)
    ODSL_beta2_mean = np.nanmean(np.array(ODSL_beta2),axis=0)

    fig,ax = plt.subplots(1,1,figsize=(10,6))

    ax.plot(ODSL_mods_mean,                 label='CMIP Data')
    ax.plot(ODSL_alph_mean,     c='g',      label=r'$\alpha$')
    ax.plot(ODSL_beta1_mean,    c='k',      label=r'$\beta_1 \times$'f'{varx1}')
    ax.plot(ODSL_beta2_mean,    c='orange', label=r'$\beta_2 \times$'f'{varx2}')
    ax.plot(ODSL_fits_mean,     c='r',      label='Regression Model')
    
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y', labelsize=15)
    ax.set_xlabel('Time', fontsize=16)
    ax.set_ylabel('ODSL change (cm)', fontsize = 16)
    ax.set_ylim([-10,40])
    ax.axvspan(0, 50, alpha=0.1, color='grey')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper left',fontsize=16)
    ax.set_title(f'{cmip} ensemble average \n Average RMSE: {np.mean(LR_result.mse.values):.2f} cm', fontsize=17)

    if cmip == 'CMIP5':
        ax.set_xlim([0,390])
        ax.set_xticks([0, 105, 200,295], ["Historic", "RCP2.6", "RCP4.5",'RCP8.5'],  horizontalalignment='left')
    elif cmip =='CMIP6':
        ax.set_xlim([0,372])
        ax.set_xticks([0, 114, 200,286], ["Historic", "SSP1-2.6", "SSP2-4.5",'SSP5-8.5'],  horizontalalignment='left')

    if save == True:
        fig.savefig(f'Figures/LR/Average_LR_{cmip}_{varx1}_{varx2}.pdf',dpi=dpi)
    elif save:
        fig.savefig(f'Figures/{save}.pdf',dpi=dpi)

    return

In [ ]:
def plot_LR_parameters(varx1, titlestr, savename=None):
    
    # open LR_results:
    LR_result_data_C5 = pd.read_csv(f'Results_LR/LR_CMIP5_{varx1}.csv')
    LR_result_C5 = LR_result_data_C5.set_index('model')
    LR_result_data_C6 = pd.read_csv(f'Results_LR/LR_CMIP6_{varx1}.csv')
    LR_result_C6 = LR_result_data_C6.set_index('model')

    labels = ['CMIP5','CMIP6']
    colors = ['b','r']

    xlabels = {'GSAT':r'[cm/$\degree$C]',
              'GMTSL' :'',
              'AMOC35':'[cm/Sv]',
              'AMOC26':'[cm/Sv]'}
    xlimits = {'GSAT':[-2,12],
              'GMTSL' :[-.5,2],
              'AMOC35':[-6,2],
              'AMOC26':[-6,2]}
    widths = {'GSAT':1,
              'GMTSL' :.2,
              'AMOC35':.5,
              'AMOC26':.5}
    ylims = [0,10]
    fig, ax = plt.subplots(1,figsize=(6.4,4),dpi=dpi)
    
    for i, pars in enumerate([LR_result_C5, LR_result_C6]):
        meanbet = pars['beta'].mean()
        w = widths[varx1]
        ax.hist(pars['beta'],bins=np.arange(np.round(min(pars['beta']))-1, max(pars['beta']) + w, w),alpha=0.2,color=colors[i],
                label=f'{labels[i]}')
        ax.axvline(meanbet,c=colors[i],ls='dashed',lw=2,label=f'Average {labels[i]}')
        ax.text(xlimits[varx1][0],9-2*i,r'  $\overline{\beta_1} =$ 'f'{pars.beta.mean():.2f}',  color = colors[i], fontsize=13)
        ax.text(xlimits[varx1][0],8-2*i,r'  $\sigma_{\beta_1} =$ 'f'{pars.beta.std():.2f}',     color = colors[i], fontsize=13)
    ax.set_xlabel(r'$\beta_1$'f' {xlabels[varx1]}',fontsize=16)
    ax.set_ylabel('Frequency',fontsize=16)
    ax.set_ylim(ylims)
    ax.set_xlim(xlimits[varx1])
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='both', labelsize=16)
    ax.legend(loc=1,fontsize=13)

    fig.suptitle(titlestr, fontsize=16,y=0.93)    
    fig.tight_layout()

    if savename is not None:
        print('saving figure')
        fig.savefig(savename,bbox_inches='tight',dpi=dpi) 


def plot_LR_parameters_multi(varx1, varx2, titlestr, savename=None):
    
    # open LR_results:
    LR_result_data_C5 = pd.read_csv(f'Results_LR/LR_CMIP5_{varx1}_{varx2}.csv')
    LR_result_C5 = LR_result_data_C5.set_index('model')
    LR_result_data_C6 = pd.read_csv(f'Results_LR/LR_CMIP6_{varx1}_{varx2}.csv')
    LR_result_C6 = LR_result_data_C6.set_index('model')

    labels = ['CMIP5','CMIP6']
    colors = ['b','r']

    xlabels = {'GSAT':r'[cm/$\degree$C]',
              'GMTSL' :'',
              'AMOC35':'[cm/Sv]',
              'AMOC26':'[cm/Sv]'}
    xlimits = {'GSAT':[-6,10],
              'GMTSL' :[-1,2],
              'AMOC35':[-6,2],
              'AMOC26':[-6,2]}
    widths = {'GSAT':1,
              'GMTSL' :.2,
              'AMOC35':.5,
              'AMOC26':.5}
    ylims = [0,10]

    fig, ax = plt.subplots(1,2, figsize=(12,4), sharey=True, dpi=dpi)
    fig.subplots_adjust(wspace=1/9)

    for i, pars in enumerate([LR_result_C5, LR_result_C6]):
        meanbet1 = pars['beta1'].mean()
        meanbet2 = pars['beta2'].mean()
        w1 = widths[varx1]
        w2 = widths[varx2]
        ax[0].hist(pars['beta1'],bins=np.arange(np.round(min(pars['beta1']))-1, max(pars['beta1']) + w1, w1),alpha=0.2,color=colors[i],
                label=f'{labels[i]}')
        ax[0].axvline(meanbet1,c=colors[i],ls='dashed',lw=2,label=f'Average {labels[i]}')
        ax[0].text(xlimits[varx1][0],9-2*i,r'  $\overline{\beta_1} =$ 'f'{pars.beta1.mean():.2f}',  color = colors[i], fontsize=13)
        ax[0].text(xlimits[varx1][0],8-2*i,r'  $\sigma_{\beta_1} =$ 'f'{pars.beta1.std():.2f}',     color = colors[i], fontsize=13)
        
        ax[1].hist(pars['beta2'],bins=np.arange(np.round(min(pars['beta2']))-1, max(pars['beta2']) + w2, w2),alpha=0.2,color=colors[i],
                label=f'{labels[i]}')
        ax[1].axvline(meanbet2,c=colors[i],ls='dashed',lw=2,label=f'Average {labels[i]}')
        ax[1].text(xlimits[varx2][0],9-2*i,r'  $\overline{\beta_2} =$ 'f'{pars.beta2.mean():.2f}',  color = colors[i], fontsize=13)
        ax[1].text(xlimits[varx2][0],8-2*i,r'  $\sigma_{\beta_2} =$ 'f'{pars.beta2.std():.2f}',     color = colors[i], fontsize=13)

    ax[0].set_xlabel(r'$\beta_1$'f' {xlabels[varx1]}',fontsize=16)
    ax[0].set_xlim(xlimits[varx1])
    ax[0].tick_params(axis='both', labelsize=16)
    ax[0].grid(True, alpha=0.3)
    
    ax[1].set_xlabel(r'$\beta_2$'f' {xlabels[varx2]}',fontsize=16)
    ax[1].set_xlim(xlimits[varx2]) 
    ax[1].tick_params(axis='both', labelsize=16)
    ax[1].grid(True, alpha=0.3)
    
    ax[0].legend(loc=0,fontsize=13)
    ax[0].set_ylabel('Frequency',fontsize=16)
    ax[0].set_ylim(ylims)   
    
    fig.suptitle(titlestr, fontsize=16,y=0.93)    
    fig.tight_layout()

    if savename is not None:
        print('saving figure')
        fig.savefig(savename,bbox_inches='tight',dpi=dpi) 

def plot_LR_parameters_multi_scatter(varx1, varx2, titlestr, savename=None):
    
    # open LR_results:
    LR_result_data_C5 = pd.read_csv(f'Results_LR/LR_CMIP5_{varx1}_{varx2}.csv')
    LR_result_C5 = LR_result_data_C5.set_index('model')
    LR_result_data_C6 = pd.read_csv(f'Results_LR/LR_CMIP6_{varx1}_{varx2}.csv')
    LR_result_C6 = LR_result_data_C6.set_index('model')

    labels = ['CMIP5','CMIP6']
    colors = ['b','r']

    xlabels = {'GSAT':r'[cm/$\degree$C]',
              'GMTSL' :'',
              'AMOC35':'[cm/Sv]',
              'AMOC26':'[cm/Sv]'}
    xlimits = {'GSAT':[-6,10],
              'GMTSL' :[-1,2],
              'AMOC35':[-6,2],
              'AMOC26':[-6,2]}

    fig, ax = plt.subplots(1,1, figsize=(4,4), dpi=dpi)

    ax.scatter(LR_result_C5['beta1'], LR_result_C5['beta2'], label='CMIP5', color='b')
    ax.scatter(LR_result_C6['beta1'], LR_result_C6['beta2'], label='CMIP6', color='r')

    ax.set_xlabel(r'$\beta_1$'f' {xlabels[varx1]}',fontsize=16)
    ax.set_ylabel(r'$\beta_2$'f' {xlabels[varx2]}',fontsize=16)

    ax.set_xlim(xlimits[varx1])
    ax.set_ylim(xlimits[varx2])

    ax.tick_params(axis='both', labelsize=16)
    ax.grid(True, alpha=0.3)

    
    ax.legend(loc='lower right',fontsize=13)
    
    fig.tight_layout()

    if savename is not None:
        print('saving figure')
        fig.savefig(savename,bbox_inches='tight',dpi=dpi) 

In [ ]:
plot_LR_parameters('GSAT',      '', savename='Figures/Fig3c.pdf')
plot_LR_parameters('GMTSL',     '', savename='Figures/Fig4c.pdf')
plot_LR_parameters('GSAT',      '', savename='Figures/LR/Params_LR_COMPARE_GSAT.pdf')
plot_LR_parameters('GMTSL',     '', savename='Figures/LR/Params_LR_COMPARE_GMTSL.pdf')
plot_LR_parameters('AMOC35',    '', savename='Figures/LR/Params_LR_COMPARE_AMOC35.pdf')

In [ ]:
plot_LR_parameters_multi_scatter('GSAT','AMOC35','', savename='Figures/Fig5d.pdf')

In [ ]:
plot_LR_parameters_multi('GSAT',    'AMOC35',   '', savename='Figures/Fig5c.pdf')
plot_LR_parameters_multi('GSAT',    'GMTSL',    '', savename='Figures/LR/Params_LR_COMPARE_GSAT_GMTSL.pdf')
plot_LR_parameters_multi('GMTSL',   'AMOC35',   '', savename='Figures/LR/Params_LR_COMPARE_GMTSL_AMOC35.pdf')
plot_LR_parameters_multi('GSAT',    'AMOC35',   '', savename='Figures/LR/Params_LR_COMPARE_GSAT_AMOC35.pdf')

In [ ]:
plot_LR_results(CMIP5_time_series,'GSAT',   'CMIP5', save=True)
plot_LR_results(CMIP5_time_series,'GMTSL',  'CMIP5', save=True)
plot_LR_results(CMIP5_time_series,'AMOC35', 'CMIP5', save=True)

plot_LR_results(CMIP6_time_series,'GSAT',   'CMIP6', save=True)
plot_LR_results(CMIP6_time_series,'GMTSL',  'CMIP6', save=True)
plot_LR_results(CMIP6_time_series,'AMOC35', 'CMIP6', save=True)

In [ ]:
plot_LR_results_multi(CMIP5_time_series,'GSAT','GMTSL',  'CMIP5', True)
plot_LR_results_multi(CMIP5_time_series,'GSAT','AMOC35', 'CMIP5', True)
plot_LR_results_multi(CMIP5_time_series,'GMTSL','AMOC35','CMIP5', True)

plot_LR_results_multi(CMIP6_time_series,'GSAT','GMTSL',  'CMIP6', True)
plot_LR_results_multi(CMIP6_time_series,'GSAT','AMOC35', 'CMIP6', True)
plot_LR_results_multi(CMIP6_time_series,'GMTSL','AMOC35','CMIP6', True)

## Figure 5. Spatial MLD

In [ ]:
CMIP5_MLD = xr.open_dataset('Data/CMIP5_spatial_MLD.nc')
CMIP6_MLD = xr.open_dataset('Data/CMIP6_spatial_MLD.nc')

In [ ]:
Regional_MLD = xr.concat([CMIP5_MLD.assign_coords({'CMIP': 'CMIP5' }).median(dim='model'), CMIP6_MLD.assign_coords({'CMIP': 'CMIP6' }).median(dim='model')],dim='CMIP')

# Plotting extent
lon_min, lon_max, lat_min, lat_max = -60.5, 40.5, 39.9999, 81

fig, ax = plt.subplots(1,2, figsize=(20,10),
            subplot_kw=dict(projection=ccrs.Robinson(), facecolor="lightgray"),
            transform=ccrs.PlateCarree(), dpi=dpi)
plt.subplots_adjust(wspace=0.02, bottom=0.15)

titles = ['a) CMIP5', 'b) CMIP6']

for i in range(2):
    cb = ax[i].pcolormesh(Regional_MLD.lon,Regional_MLD.lat,Regional_MLD.isel(CMIP = i).MLD.values,transform = ccrs.PlateCarree(),vmin=0,vmax=400,cmap='cmo.dense')
    ax[i].add_feature(cfeature.LAND, facecolor='lightgrey', zorder=1) 
    ax[i].coastlines()   
    ax[i].set_title(titles[i],fontsize=22)
    ax[i].set_extent([lon_min, lon_max, lat_min, lat_max])

    gl = ax[i].gridlines(color='grey', linestyle='--', alpha=0.4, draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False
    gl.xlocator = mticker.FixedLocator([-80,-40,0,40])
    gl.ylocator = mticker.FixedLocator([40,60,80])

    gl.xlabel_style = {'size': 22, 'color':'black'}
    gl.ylabel_style = {'size': 22, 'color':'black'}

    # Add ocean boxes
    box_colors = ['k','r','b']
    box_labels = ['Labrador Sea', 'Greenland Sea', 'Irminger Sea']
    for j, box in enumerate(['LS', 'GS', 'IS']):
        lon_min_b, lon_max_b, lat_min_b, lat_max_b = define_area(box)
        x, y = ([lon_min_b, lon_min_b, lon_max_b, lon_max_b, lon_min_b], 
            [lat_max_b, lat_min_b, lat_min_b, lat_max_b, lat_max_b])

        ax[i].plot(x, y, marker='o', color = box_colors[j], linewidth=4.0, transform=ccrs.PlateCarree(), label = box_labels[j])

# Show ocean box legend in CMIP5 plot
ax[0].legend(loc='lower right', fontsize=20)

# Add colorbar
cax = fig.add_axes([.15,.124,1,.754])
cax.set_axis_off()
cbar = plt.colorbar(cb, ax = cax,cmap='cmo.balance', orientation='vertical',ticklocation='auto',extend='max',shrink=0.53)
cbar.ax.tick_params(labelsize=20) 
cbar.set_label('MLD [m]', size=20)

fig.tight_layout();
fig.savefig('Figures/Fig2a-b.pdf', bbox_inches='tight', dpi = dpi);

## Figure 6. MLD Results
- Different regions
- GS threshold 170 m

In [ ]:
CMIP5_MLD_stats = pd.read_csv('Results_MLD/Regional_MLD_CMIP5.csv').set_index('Model')
CMIP6_MLD_stats = pd.read_csv('Results_MLD/Regional_MLD_CMIP6.csv').set_index('Model')

In [ ]:
CMIP5_LS_models = CMIP5_MLD_stats.loc[CMIP5_MLD_stats.region_mean == 'LS_mean'].index.values
CMIP5_GS_models = CMIP5_MLD_stats.loc[CMIP5_MLD_stats.region_mean == 'GS_mean'].index.values
CMIP5_IS_models = CMIP5_MLD_stats.loc[CMIP5_MLD_stats.region_mean == 'IS_mean'].index.values

CMIP6_LS_models = CMIP6_MLD_stats.loc[CMIP6_MLD_stats.region_mean == 'LS_mean'].index.values
CMIP6_GS_models = CMIP6_MLD_stats.loc[CMIP6_MLD_stats.region_mean == 'GS_mean'].index.values
CMIP6_IS_models = CMIP6_MLD_stats.loc[CMIP6_MLD_stats.region_mean == 'IS_mean'].index.values

In [ ]:
def plot_zos_per_MLD_region(CMIP5_ts, CMIP6_ts, CMIP5_MLD_st, CMIP6_MLD_st, CMIP5_scenario, CMIP6_scenario, titles, savename=None):

    fig, ax = plt.subplots(1,4,figsize=(14,6),gridspec_kw={'width_ratios': [8, 1, 8, 1]},sharey=True,dpi=dpi)
    plt.subplots_adjust(wspace=0.02, bottom=0.15)

    MLD_stats_set = [CMIP5_MLD_st, CMIP6_MLD_st]
    scenarios = [CMIP5_scenario, CMIP6_scenario]

    for i, CMIP_time_series in enumerate([CMIP5_ts, CMIP6_ts]):
        MLD_stats = MLD_stats_set[i]
        emission_scenario = scenarios[i]
        
        time = CMIP_time_series.time.values
        zos = CMIP_time_series.sel(scenario=emission_scenario).zos

        CMIP_LS_models = MLD_stats.loc[MLD_stats.region_mean == 'LS_mean'].index.values
        CMIP_GS_models = MLD_stats.loc[MLD_stats.region_mean == 'GS_mean'].index.values
        CMIP_IS_models = MLD_stats.loc[MLD_stats.region_mean == 'IS_mean'].index.values

        for j in range(len(CMIP_LS_models)):
            ax[i*2].plot(time, zos.sel(model=CMIP_LS_models).values[j,:], c='k', alpha=0.2)
        for j in range(len(CMIP_GS_models)):
            ax[i*2].plot(time, zos.sel(model=CMIP_GS_models).values[j,:], c='r', alpha=0.2)
        for j in range(len(CMIP_IS_models)):
            ax[i*2].plot(time, zos.sel(model=CMIP_IS_models).values[j,:], c='b', alpha=0.2)

        # Plot median
        LS_med, = ax[i*2].plot(time, zos.sel(model=CMIP_LS_models).median(dim='model'), c='k', lw=3, label='LS models')
        GS_med, = ax[i*2].plot(time, zos.sel(model=CMIP_GS_models).median(dim='model'), c='r', lw=3, label='GS models')
        IS_med, = ax[i*2].plot(time, zos.sel(model=CMIP_IS_models).median(dim='model'), c='b', lw=3, label='IS models')

        ax[i*2].set_xlim(1900,2100)
        ax[i*2].axvspan(1900,1950,color='grey',alpha=0.1)
        ax[i*2].set_ylim([-10,40])
        ax[i*2].set_xlabel('Year', fontsize = 20)
        ax[i*2].tick_params(axis='both', labelsize= 18, which='major', pad=10)
        ax[i*2].grid(True,alpha=0.3)
        ax[i*2].set_yticks([-10,0,10,20,30,40])
        ax[i*2].set_title(titles[i], fontsize=20);
        
        ax[i*2+1].set_axis_off()
        ax[i*2+1].set_xlim(-0.5,3.0)
        
    ax[0].legend([LS_med, GS_med, IS_med], 
             ["LS models", "GS models", "IS models"],
             loc=2, fontsize=16);
    ax[0].set_ylabel('ODSL change (cm)', fontsize = 20)

    if savename:
        fig.savefig(f'Figures/{savename}', bbox_inches='tight', dpi = dpi)    
    return 

In [ ]:
def make_barplot_MLD(CMIP_time_series, CMIP_models, CMIP_scenarios, perc, start_year, end_year):

    # Create Data Frame 
    col = [f'Scenario']
    for p in perc:
        col.append(f'Percentile: {p}')    
    summary_df = pd.DataFrame(columns=col)
    
    sces = CMIP_scenarios
    
    for idx,sce in enumerate(sces):
        # Check models:
        ds_sce = CMIP_time_series.sel(scenario=sce)
        ds_mods = ds_sce['zos'].dropna('model','all')
        ds_models = list(set(ds_mods.model.values)&set(CMIP_models))
        ds_mods = ds_mods.sel(model=ds_models)

        # Calculate zos for period of interest:
        zos = ds_mods.sel(time=slice(start_year, end_year)).mean(dim='time')

        # Append the percentile scores to a list val
        val = [sces[idx]]
        for p in perc:
            val.append(round(np.quantile(zos.values,p/100),1))
        summary_df.loc[idx] = val
    
    summary_df.set_index(col[0], inplace=True)
    summary_df = summary_df.T
        
    return summary_df

def plotMLD(ax,df,colors):
    mi = 0.6 # Max color intensity
        
    if colors=='linspace':
        colors1 = plt.cm.Reds(np.linspace(0, mi, len(df.index)))
        colors2 = plt.cm.Blues(np.linspace(0, mi, len(df.index)))
        colors = np.zeros([len(df.columns), len(df.index), 4])
        colors[::2] = colors2
        colors[1::2] = colors1
        
    elif colors=='max':
        colors1 = plt.cm.Reds([0,0.3,0.6,0.6,0.3])
        colors2 = plt.cm.Greys([0,0.3,0.6,0.6,0.3])
        colors = np.zeros([len(df.columns), len(df.index), 4])
        colors[::2] = colors2
        colors[1::2] = colors1

    else:
        print('Choose `max` or `linspace` for colors or add a new option')

    # Start from white color
    colors[:,0,:] = 0
    
    index = np.arange(len(df.columns))
    bar_width = [0.1,0.2,0.4,0.4,0.2]

    # Initialize the vertical-offset for the stacked bar chart.
    y_offset = np.zeros(len(df.columns))

    # Plot bars and create text labels for the table
    cell_text = []
    for row in range(len(df.index)):
        ax.bar(index, 
               df.iloc[row]-y_offset, 
               bar_width[row], 
               bottom=y_offset, 
               color=colors[:,row,:])
        
        y_offset = df.iloc[row]
        cell_text.append(['%1.1f' % x for x in df.iloc[row]])
    
    ax.set_xlim(-0.5,index[-1]+1.5)

    # Visualize median by a white horizontal line
    ax.hlines(df.iloc[2,0], xmin=index[0]-0.7*bar_width[2], xmax=index[0]+0.7*bar_width[2], lw=2, color='white', zorder=5)
    ax.hlines(df.iloc[2,1], xmin=index[1]-0.7*bar_width[2], xmax=index[1]+0.7*bar_width[2], lw=2, color='white', zorder=5)

    return ax

In [ ]:
def plot_zos_for_GS_threshold(CMIP5_ts, CMIP6_ts, CMIP5_MLD_st, CMIP6_MLD_st, CMIP5_scenario, CMIP6_scenario, titles, savename=None):

    fig, ax = plt.subplots(1,4,figsize=(14,6),gridspec_kw={'width_ratios': [8, 1, 8, 1]},sharey=True,dpi=dpi)
    plt.subplots_adjust(wspace=0.02, bottom=0.15)

    MLD_stats_set = [CMIP5_MLD_st, CMIP6_MLD_st]
    scenarios = [CMIP5_scenario, CMIP6_scenario]

    for i, CMIP_time_series in enumerate([CMIP5_ts, CMIP6_ts]):
        MLD_stats = MLD_stats_set[i]
        emission_scenario = scenarios[i]

        time = CMIP_time_series.time.values
        zos = CMIP_time_series.sel(scenario=emission_scenario).zos

        CMIP_zos_mods = zos.dropna('model','all')['model'].values
        
        CMIP_GS_models = MLD_stats.loc[MLD_stats['GS_mean>170'] == True].index.values
        CMIP_other_models = MLD_stats.loc[MLD_stats['GS_mean>170'] == False].index.values

        for j in range(len(CMIP_GS_models)):
            ax[i*2].plot(time, zos.sel(model=CMIP_GS_models).values[j,:], c='r', alpha=0.2)
        for j in range(len(CMIP_other_models)):
            ax[i*2].plot(time, zos.sel(model=CMIP_other_models).values[j,:], c='grey', alpha=0.2)

        # Plot median
        GS_med, = ax[i*2].plot(time, zos.sel(model=CMIP_GS_models).median(dim='model'), c='r', lw=3, label='GS models')
        ot_med, = ax[i*2].plot(time, zos.sel(model=CMIP_other_models).median(dim='model'), c='grey', lw=3, label='Other models')

        ax[i*2].set_xlim(1900,2100)
        ax[i*2].axvspan(1900,1950,color='grey',alpha=0.1)
        ax[i*2].set_ylim([-10,40])
        ax[i*2].set_xlabel('Year', fontsize = 20)
        ax[i*2].tick_params(axis='both', labelsize= 18, which='major', pad=10)
        ax[i*2].grid(True,alpha=0.3)
        ax[i*2].set_yticks([-10,0,10,20,30,40])
        ax[i*2].set_title(titles[i], fontsize=20);
        #ax[i*2].legend(True)
        # Plot bars
        percentiles = [5, 17, 50, 83, 95]
        df_GS = make_barplot_MLD(CMIP_time_series, CMIP_time_series.sel(model=CMIP_GS_models).model.values, [emission_scenario], percentiles, 2081, 2101)
        df_other = make_barplot_MLD(CMIP_time_series, CMIP_time_series.sel(model=CMIP_other_models).model.values, [emission_scenario], percentiles, 2081, 2101)
        df = pd.concat([df_other,df_GS],axis=1)

        ax[i*2+1].set_axis_off()
        ax[i*2+1] = plotMLD(ax[i*2+1], df, colors='max')
        ax[i*2+1].text(0,   df.iloc[0,0]-2,f'{len(list(set(CMIP_other_models)&set(CMIP_zos_mods)))}',ha='center')
        ax[i*2+1].text(1.0, df.iloc[0,1]-2,f'{len(list(set(CMIP_GS_models)&set(CMIP_zos_mods)))}',ha='center',color='red')
        ax[i*2+1].set_xlim(-0.5,3.0)
    
    ax[0].legend([GS_med, ot_med], 
             ["GS models", "Other"],
             loc=2, fontsize=16);
    ax[0].set_ylabel('ODSL change (cm)', fontsize = 20)
    if savename:
        fig.savefig(f'Figures/{savename}', bbox_inches='tight', dpi = dpi)

    return 

In [ ]:
plot_zos_per_MLD_region(CMIP5_time_series, CMIP6_time_series, CMIP5_MLD_stats, CMIP6_MLD_stats, 'rcp26', 'ssp126', titles=['a) CMIP5 RCP2.6','b) CMIP6 SSP1-2.6'],savename='Appendix/MLD_low.pdf')
plot_zos_per_MLD_region(CMIP5_time_series, CMIP6_time_series, CMIP5_MLD_stats, CMIP6_MLD_stats, 'rcp45', 'ssp245', titles=['a) CMIP5 RCP4.5','b) CMIP6 SSP2-4.5'],savename='Fig6a-b.pdf')
plot_zos_per_MLD_region(CMIP5_time_series, CMIP6_time_series, CMIP5_MLD_stats, CMIP6_MLD_stats, 'rcp85', 'ssp585', titles=['a) CMIP5 RCP8.5','b) CMIP6 SSP5-8.5'],savename='Appendix/MLD_high.pdf')

In [ ]:
plot_zos_for_GS_threshold(CMIP5_time_series, CMIP6_time_series, CMIP5_MLD_stats, CMIP6_MLD_stats, 'rcp26', 'ssp126', titles=['c) CMIP5 RCP2.6','d) CMIP6 SSP1-2.6'],savename='Appendix/MLD_GS_low.pdf')
plot_zos_for_GS_threshold(CMIP5_time_series, CMIP6_time_series, CMIP5_MLD_stats, CMIP6_MLD_stats, 'rcp45', 'ssp245', titles=['c) CMIP5 RCP4.5','d) CMIP6 SSP2-4.5'],savename='Fig6c-d.pdf')
plot_zos_for_GS_threshold(CMIP5_time_series, CMIP6_time_series, CMIP5_MLD_stats, CMIP6_MLD_stats, 'rcp85', 'ssp585', titles=['c) CMIP5 RCP8.5','d) CMIP6 SSP5-8.5'],savename='Appendix/MLD_GS_high.pdf')